# Module 07: Position Sizing and Risk Management

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 85 minutes

**Prerequisites**: 
- Completed Module 00: Setup and Introduction
- Completed Module 01: Bursa Malaysia Fundamentals
- Completed Module 02: Data Collection with yfinance
- Completed Module 03: Moving Averages and Trends
- Completed Module 04: RSI and MACD Indicators
- Completed Module 05: Chart Patterns and Volume Analysis
- Completed Module 06: Entry and Exit Strategies

## Learning Objectives

By the end of this notebook, you will be able to:
1. Apply the 2% rule to limit risk per trade
2. Calculate position sizes based on account size and stop-loss distance
3. Use ATR-based position sizing for volatility adjustment
4. Apply Kelly Criterion for optimal position sizing
5. Design portfolio allocation strategies (core/swing/opportunistic)
6. Implement drawdown management protocols
7. Apply diversification rules to manage correlation risk

## Introduction: Why Position Sizing Matters More Than Entry/Exit

### The Shocking Truth

**Position sizing determines 90% of your long-term returns** - not your entry timing, not your exit strategy.

### Real Example

**Two traders, same strategy, different position sizing**:

**Trader A** (No position sizing):
- Buys RM10,000 worth every trade
- Stop-loss varies: 3% to 10%
- Risk varies: RM300 to RM1,000 per trade
- One 10% loss = RM1,000 (needs 3-4 wins to recover)
- **Result**: Inconsistent, high stress, eventual blow-up

**Trader B** (2% rule):
- Risks exactly 2% per trade (RM200 on RM10,000 account)
- Adjusts position size based on stop distance
- Wide stop = smaller position
- Tight stop = larger position
- **Result**: Consistent risk, steady growth, survives bad streaks

### The Professional Approach

Professional traders:
1. ✅ **First**: Calculate position size
2. ✅ **Second**: Find entry
3. ✅ **Third**: Set stop-loss
4. ✅ **Fourth**: Execute trade

Amateur traders:
1. ❌ Find entry
2. ❌ Buy as much as they can afford
3. ❌ Hope it goes up
4. ❌ Set stop-loss (maybe)

### Malaysian Market Context

**Typical Bursa Malaysia retail investor mistakes**:
- Putting 20-50% of capital in one trade
- No stop-losses ("I'll average down")
- Overtrading low-conviction setups
- Revenge trading after losses

**Professional approach for Malaysia**:
- 2% risk per trade maximum
- Position size based on stop distance
- Core holdings: 40% of portfolio
- Swing trades: 30% of portfolio
- Opportunistic: 20% of portfolio
- Cash buffer: 10% minimum

Let's master professional position sizing!

In [ ]:
# Setup: Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Visualization configuration
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 7)

np.random.seed(42)

print("✅ Environment setup complete!")

## 1. The 2% Rule: Foundation of Risk Management

### What is the 2% Rule?

**Never risk more than 2% of your account on a single trade.**

### Why 2%?

**Survival math**:
- 10 consecutive losses at 2% = 18% total loss (survivable)
- 10 consecutive losses at 10% = 65% total loss (devastating)
- 10 consecutive losses at 20% = 89% total loss (nearly wiped out)

**Recovery math**:
- 10% loss requires 11% gain to recover
- 20% loss requires 25% gain to recover
- 50% loss requires 100% gain to recover
- 80% loss requires 400% gain to recover

### The 2% Rule Formula

```
Maximum Risk Per Trade = Account Size × 0.02
```

**Examples**:
- RM10,000 account → RM200 max risk per trade
- RM50,000 account → RM1,000 max risk per trade
- RM100,000 account → RM2,000 max risk per trade

### Risk Tiers

| Risk Level | % Per Trade | Use Case |
|-----------|-------------|----------|
| **Ultra Conservative** | 0.5-1% | Beginners, large accounts |
| **Conservative** | 1-1.5% | Risk-averse, retirement accounts |
| **Standard** | 2% | Most traders, recommended |
| **Aggressive** | 2.5-3% | Experienced, high-conviction |
| **Reckless** | 5%+ | Gambling, not recommended |

### Malaysian Market Context

For **Bursa Malaysia retail investors**:
- Start with 1% if account < RM10,000
- Use 2% for accounts RM10,000-RM100,000
- Consider 1.5% for very volatile small caps
- Never exceed 3% even on "sure things"

Let's see the 2% rule in action!

In [ ]:
# Demonstrate the 2% rule with examples

def calculate_max_risk(account_size, risk_percentage=0.02):
    """
    Calculate maximum risk per trade based on 2% rule.
    
    Args:
        account_size (float): Total account value in RM
        risk_percentage (float): Risk per trade (0.02 = 2%)
    
    Returns:
        float: Maximum risk amount
    """
    return account_size * risk_percentage

# Example account sizes
account_sizes = [5000, 10000, 25000, 50000, 100000]
risk_levels = {
    'Conservative (1%)': 0.01,
    'Standard (2%)': 0.02,
    'Aggressive (3%)': 0.03
}

print("Maximum Risk Per Trade (2% Rule):")
print("=" * 80)
print(f"\n{'Account Size':<15} {'1% Risk':<12} {'2% Risk':<12} {'3% Risk':<12}")
print("-" * 80)

for size in account_sizes:
    risk_1pct = calculate_max_risk(size, 0.01)
    risk_2pct = calculate_max_risk(size, 0.02)
    risk_3pct = calculate_max_risk(size, 0.03)
    
    print(f"RM{size:>12,}  RM{risk_1pct:>9,.0f}  RM{risk_2pct:>9,.0f}  RM{risk_3pct:>9,.0f}")

print("\n💡 Key Insight: Risk amount scales with account size")
print("   Larger account = Larger dollar risk, but same percentage risk")

# Demonstrate survival through losing streaks
print("\n\nLosing Streak Survival Analysis:")
print("=" * 80)

account = 10000
consecutive_losses = 10

print(f"Starting Account: RM{account:,.0f}")
print(f"Consecutive Losses: {consecutive_losses}\n")

for risk_name, risk_pct in risk_levels.items():
    balance = account
    
    for i in range(consecutive_losses):
        loss = balance * risk_pct
        balance -= loss
    
    total_loss_pct = ((account - balance) / account) * 100
    recovery_needed = ((account - balance) / balance) * 100
    
    status = "✅ Survivable" if total_loss_pct < 30 else "⚠️ Dangerous" if total_loss_pct < 60 else "❌ Devastating"
    
    print(f"{risk_name}:")
    print(f"  Final Balance:     RM{balance:>9,.0f}")
    print(f"  Total Loss:        {total_loss_pct:>6.1f}%")
    print(f"  Recovery Needed:   {recovery_needed:>6.1f}%")
    print(f"  Status:            {status}\n")

print("💡 The 2% rule keeps you in the game even during terrible losing streaks!")

## 2. Position Size Calculation: The Core Formula

### The Position Sizing Formula

```
Position Size = Risk Amount / Stop-Loss Distance
```

Where:
- **Risk Amount** = Account Size × Risk %  (from 2% rule)
- **Stop-Loss Distance** = Entry Price - Stop Price

### Step-by-Step Example

**Given**:
- Account Size: RM10,000
- Risk: 2% = RM200
- Entry Price: RM10.00
- Stop-Loss: RM9.50

**Calculate**:
1. Stop Distance = RM10.00 - RM9.50 = RM0.50
2. Position Size = RM200 / RM0.50 = 400 shares
3. Position Value = 400 × RM10.00 = RM4,000

**Verification**:
- If stopped out: 400 shares × RM0.50 loss = RM200 loss ✅
- Risk = RM200 / RM10,000 = 2% ✅

### Why This Works

**Wide stop-loss**:
- Large stop distance
- Smaller position size
- Still risk only 2%

**Tight stop-loss**:
- Small stop distance
- Larger position size
- Still risk only 2%

**Result**: Consistent risk regardless of stop placement!

### Malaysian Market Considerations

**Minimum lot sizes**:
- Most stocks: 100 shares minimum
- Must round to nearest 100

**Brokerage fees**:
- Typical: 0.42% (RM2.88 minimum)
- Factor into calculations for small positions

Let's implement position size calculations!

In [ ]:
# Position size calculator

def calculate_position_size(account_size, risk_pct, entry_price, stop_price, 
                           lot_size=100, min_fee=2.88, fee_pct=0.0042):
    """
    Calculate position size based on risk management rules.
    
    Args:
        account_size (float): Total account value
        risk_pct (float): Risk percentage (0.02 = 2%)
        entry_price (float): Entry price per share
        stop_price (float): Stop-loss price per share
        lot_size (int): Minimum lot size (usually 100)
        min_fee (float): Minimum brokerage fee
        fee_pct (float): Brokerage fee percentage
    
    Returns:
        dict: Position sizing details
    """
    # Calculate risk amount
    risk_amount = account_size * risk_pct
    
    # Calculate stop distance
    stop_distance = entry_price - stop_price
    stop_distance_pct = (stop_distance / entry_price) * 100
    
    # Calculate ideal position size
    ideal_shares = risk_amount / stop_distance
    
    # Round to nearest lot
    actual_shares = round(ideal_shares / lot_size) * lot_size
    
    # Calculate position value
    position_value = actual_shares * entry_price
    
    # Calculate actual risk
    actual_risk = actual_shares * stop_distance
    actual_risk_pct = (actual_risk / account_size) * 100
    
    # Calculate brokerage fees (buy + sell)
    buy_fee = max(position_value * fee_pct, min_fee)
    sell_fee = max((actual_shares * stop_price) * fee_pct, min_fee)
    total_fees = buy_fee + sell_fee
    
    # Calculate portfolio allocation
    portfolio_pct = (position_value / account_size) * 100
    
    return {
        'account_size': account_size,
        'risk_amount': risk_amount,
        'risk_pct': risk_pct * 100,
        'entry_price': entry_price,
        'stop_price': stop_price,
        'stop_distance': stop_distance,
        'stop_distance_pct': stop_distance_pct,
        'ideal_shares': ideal_shares,
        'actual_shares': actual_shares,
        'position_value': position_value,
        'actual_risk': actual_risk,
        'actual_risk_pct': actual_risk_pct,
        'buy_fee': buy_fee,
        'sell_fee': sell_fee,
        'total_fees': total_fees,
        'portfolio_pct': portfolio_pct
    }

# Example calculations
examples = [
    {'name': 'Tight Stop (3%)', 'entry': 10.00, 'stop': 9.70},
    {'name': 'Normal Stop (5%)', 'entry': 10.00, 'stop': 9.50},
    {'name': 'Wide Stop (8%)', 'entry': 10.00, 'stop': 9.20}
]

account = 10000
risk = 0.02

print("Position Size Examples (RM10,000 account, 2% risk):")
print("=" * 90)

for ex in examples:
    result = calculate_position_size(account, risk, ex['entry'], ex['stop'])
    
    print(f"\n{ex['name']}:")
    print(f"  Entry Price:       RM{result['entry_price']:.2f}")
    print(f"  Stop Price:        RM{result['stop_price']:.2f}")
    print(f"  Stop Distance:     RM{result['stop_distance']:.2f} ({result['stop_distance_pct']:.1f}%)")
    print(f"  Position Size:     {result['actual_shares']:.0f} shares")
    print(f"  Position Value:    RM{result['position_value']:,.2f}")
    print(f"  Portfolio %:       {result['portfolio_pct']:.1f}%")
    print(f"  Actual Risk:       RM{result['actual_risk']:.2f} ({result['actual_risk_pct']:.2f}%)")
    print(f"  Total Fees:        RM{result['total_fees']:.2f}")

print("\n\n💡 Key Observations:")
print("   • Tighter stop = LARGER position size (less risk per share)")
print("   • Wider stop = SMALLER position size (more risk per share)")
print("   • Risk stays constant at ~2% regardless of stop placement")
print("   • Position value varies significantly based on stop distance")

## 3. ATR-Based Position Sizing

### What is ATR-Based Sizing?

**Average True Range (ATR)** measures volatility. ATR-based sizing adjusts position size for a stock's volatility.

### Why Use ATR?

**Different stocks have different volatility**:
- Maybank: Stable blue chip, low volatility
- Tech startups: High volatility

**ATR-based sizing**:
- High volatility stock → Wider stop → Smaller position
- Low volatility stock → Tighter stop → Larger position
- **Result**: Normalized risk across different volatility profiles

### ATR Position Sizing Formula

```
Stop Distance = ATR × Multiplier
Position Size = Risk Amount / Stop Distance
```

**Common ATR multipliers**:
- Conservative: 1.5× ATR
- Standard: 2× ATR
- Aggressive: 3× ATR

### Example

**Stock A** (Low volatility):
- Price: RM10.00
- ATR: RM0.20
- Stop: 2× ATR = RM0.40 below entry
- Position: RM200 / RM0.40 = 500 shares

**Stock B** (High volatility):
- Price: RM10.00
- ATR: RM0.80
- Stop: 2× ATR = RM1.60 below entry
- Position: RM200 / RM1.60 = 125 shares

**Result**: Same risk (RM200), but position size adapts to volatility!

Let's implement ATR-based position sizing!

In [ ]:
# Download data and calculate ATR for position sizing

def calculate_atr(data, period=14):
    """Calculate Average True Range."""
    df = data.copy()
    
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=period).mean()
    
    return df['ATR']

# Download multiple stocks with different volatility profiles
stocks = {
    '1155.KL': 'Maybank (Low Vol)',
    '1295.KL': 'Public Bank (Low Vol)',
    '5398.KL': 'Gamuda (Medium Vol)'
}

start_date = '2023-01-01'
end_date = '2024-12-31'

stock_data = {}

print("Downloading stock data for ATR analysis...\n")

for ticker, name in stocks.items():
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data['ATR'] = calculate_atr(data)
    stock_data[ticker] = {
        'name': name,
        'data': data,
        'current_price': data['Close'].iloc[-1],
        'current_atr': data['ATR'].iloc[-1]
    }
    print(f"✅ {ticker} ({name}): Price RM{data['Close'].iloc[-1]:.2f}, "
          f"ATR RM{data['ATR'].iloc[-1]:.2f}")

print("\nData downloaded successfully!")

In [ ]:
# Calculate ATR-based position sizes

def calculate_atr_position_size(account_size, risk_pct, entry_price, atr, 
                               atr_multiplier=2.0, lot_size=100):
    """
    Calculate position size using ATR-based stop-loss.
    
    Args:
        account_size (float): Total account value
        risk_pct (float): Risk percentage (0.02 = 2%)
        entry_price (float): Entry price per share
        atr (float): Current ATR value
        atr_multiplier (float): ATR multiplier for stop (2.0 = 2× ATR)
        lot_size (int): Minimum lot size
    
    Returns:
        dict: ATR-based position sizing details
    """
    # Calculate risk amount
    risk_amount = account_size * risk_pct
    
    # Calculate stop distance based on ATR
    stop_distance = atr * atr_multiplier
    stop_price = entry_price - stop_distance
    stop_distance_pct = (stop_distance / entry_price) * 100
    
    # Calculate position size
    ideal_shares = risk_amount / stop_distance
    actual_shares = round(ideal_shares / lot_size) * lot_size
    
    # Calculate position value
    position_value = actual_shares * entry_price
    
    # Calculate actual risk
    actual_risk = actual_shares * stop_distance
    actual_risk_pct = (actual_risk / account_size) * 100
    
    # Portfolio allocation
    portfolio_pct = (position_value / account_size) * 100
    
    # ATR as percentage of price (volatility measure)
    atr_pct = (atr / entry_price) * 100
    
    return {
        'entry_price': entry_price,
        'atr': atr,
        'atr_pct': atr_pct,
        'atr_multiplier': atr_multiplier,
        'stop_distance': stop_distance,
        'stop_price': stop_price,
        'stop_distance_pct': stop_distance_pct,
        'actual_shares': actual_shares,
        'position_value': position_value,
        'actual_risk': actual_risk,
        'actual_risk_pct': actual_risk_pct,
        'portfolio_pct': portfolio_pct
    }

# Calculate ATR-based positions for all stocks
account = 10000
risk = 0.02
atr_mult = 2.0

print(f"ATR-Based Position Sizing (RM{account:,} account, 2% risk, 2× ATR stop):")
print("=" * 90)

for ticker, info in stock_data.items():
    price = info['current_price']
    atr = info['current_atr']
    
    result = calculate_atr_position_size(account, risk, price, atr, atr_mult)
    
    print(f"\n{ticker} - {info['name']}:")
    print(f"  Current Price:     RM{result['entry_price']:.2f}")
    print(f"  ATR:               RM{result['atr']:.2f} ({result['atr_pct']:.2f}% of price)")
    print(f"  Stop Distance:     RM{result['stop_distance']:.2f} ({result['stop_distance_pct']:.2f}%)")
    print(f"  Stop Price:        RM{result['stop_price']:.2f}")
    print(f"  Position Size:     {result['actual_shares']:.0f} shares")
    print(f"  Position Value:    RM{result['position_value']:,.2f}")
    print(f"  Portfolio %:       {result['portfolio_pct']:.1f}%")
    print(f"  Actual Risk:       RM{result['actual_risk']:.2f} ({result['actual_risk_pct']:.2f}%)")

print("\n\n💡 Key Observations:")
print("   • Higher ATR (volatility) → Wider stop → Smaller position")
print("   • Lower ATR (stability) → Tighter stop → Larger position")
print("   • Risk stays constant at ~2% across all volatility levels")
print("   • Position size automatically adapts to market conditions")

print("\n✅ ATR-based sizing is ideal for comparing stocks with different volatilities!")

## 4. Kelly Criterion: Optimal Position Sizing

### What is Kelly Criterion?

**Kelly Criterion** calculates the mathematically optimal position size to maximize long-term growth.

### The Kelly Formula

```
Kelly % = (Win Rate × Avg Win - Loss Rate × Avg Loss) / Avg Win

Or simplified:

Kelly % = W - [(1 - W) / R]
```

Where:
- **W** = Win rate (probability of winning)
- **R** = Win/Loss ratio (average win / average loss)

### Example Calculation

**Strategy statistics**:
- Win rate: 60%
- Average win: 5%
- Average loss: 3%
- Win/Loss ratio: 5% / 3% = 1.67

**Kelly Calculation**:
```
Kelly % = 0.60 - [(1 - 0.60) / 1.67]
        = 0.60 - [0.40 / 1.67]
        = 0.60 - 0.24
        = 0.36 (36%)
```

### The Problem with Full Kelly

**36% per trade is INSANE**:
- Too aggressive
- High volatility
- Psychological stress
- Risk of ruin

### Fractional Kelly (Recommended)

Professional traders use **fractional Kelly**:

| Fraction | Aggression | Use Case |
|----------|-----------|----------|
| **1/4 Kelly** | Conservative | Most traders |
| **1/3 Kelly** | Moderate | Experienced traders |
| **1/2 Kelly** | Aggressive | High-conviction setups |
| **Full Kelly** | Reckless | Never recommended |

**Example**: If Kelly suggests 36%, use:
- 1/4 Kelly = 9% per trade
- 1/3 Kelly = 12% per trade
- 1/2 Kelly = 18% per trade

### Malaysian Market Application

For **Bursa Malaysia strategies**:
- Calculate Kelly based on backtest results
- Use 1/4 Kelly for position sizing
- Never exceed 2-3% risk even if Kelly suggests more
- Kelly as a guide, not a strict rule

Let's implement Kelly Criterion!

In [ ]:
# Kelly Criterion calculator

def calculate_kelly(win_rate, avg_win, avg_loss, fraction=0.25):
    """
    Calculate Kelly Criterion position size.
    
    Args:
        win_rate (float): Probability of winning (0.60 = 60%)
        avg_win (float): Average win percentage (0.05 = 5%)
        avg_loss (float): Average loss percentage (0.03 = 3%)
        fraction (float): Fractional Kelly (0.25 = 1/4 Kelly)
    
    Returns:
        dict: Kelly calculations
    """
    # Calculate win/loss ratio
    win_loss_ratio = avg_win / avg_loss
    
    # Calculate full Kelly percentage
    full_kelly = win_rate - ((1 - win_rate) / win_loss_ratio)
    
    # Calculate fractional Kelly
    fractional_kelly = full_kelly * fraction
    
    # Calculate expected value per trade
    expected_value = (win_rate * avg_win) - ((1 - win_rate) * avg_loss)
    
    return {
        'win_rate': win_rate * 100,
        'loss_rate': (1 - win_rate) * 100,
        'avg_win': avg_win * 100,
        'avg_loss': avg_loss * 100,
        'win_loss_ratio': win_loss_ratio,
        'full_kelly': full_kelly * 100,
        'fractional_kelly': fractional_kelly * 100,
        'fraction': fraction,
        'expected_value': expected_value * 100
    }

# Example strategies with different profiles
strategies = [
    {
        'name': 'Conservative Strategy',
        'win_rate': 0.55,
        'avg_win': 0.03,
        'avg_loss': 0.02
    },
    {
        'name': 'Balanced Strategy',
        'win_rate': 0.60,
        'avg_win': 0.05,
        'avg_loss': 0.03
    },
    {
        'name': 'Aggressive Strategy',
        'win_rate': 0.65,
        'avg_win': 0.08,
        'avg_loss': 0.04
    },
    {
        'name': 'Malaysian RSI+MACD',
        'win_rate': 0.73,  # From Module 04 research
        'avg_win': 0.0088,  # 0.88% average gain
        'avg_loss': 0.03    # Assume 3% stop-loss
    }
]

print("Kelly Criterion Analysis:")
print("=" * 90)

for strat in strategies:
    result = calculate_kelly(strat['win_rate'], strat['avg_win'], strat['avg_loss'])
    
    print(f"\n{strat['name']}:")
    print(f"  Win Rate:          {result['win_rate']:.1f}%")
    print(f"  Average Win:       {result['avg_win']:.2f}%")
    print(f"  Average Loss:      {result['avg_loss']:.2f}%")
    print(f"  Win/Loss Ratio:    {result['win_loss_ratio']:.2f}")
    print(f"  Expected Value:    {result['expected_value']:.2f}% per trade")
    print(f"  Full Kelly:        {result['full_kelly']:.2f}%")
    print(f"  1/4 Kelly:         {result['fractional_kelly']:.2f}%")
    
    # Recommendation
    if result['fractional_kelly'] <= 2:
        rec = "✅ Safe - Use 1/4 Kelly"
    elif result['fractional_kelly'] <= 5:
        rec = "⚠️ Moderate - Consider using less"
    else:
        rec = "❌ Too aggressive - Cap at 2-3%"
    
    print(f"  Recommendation:    {rec}")

print("\n\n💡 Key Insights:")
print("   • Full Kelly is usually too aggressive for retail traders")
print("   • Use 1/4 Kelly or 1/3 Kelly for reasonable position sizes")
print("   • Even with great stats, keep position sizes reasonable")
print("   • Kelly should guide, not dictate position sizing")

print("\n🇲🇾 Malaysian Market Recommendation:")
print("   • Use Kelly to compare strategies, not for exact sizing")
print("   • Never exceed 2-3% risk per trade, regardless of Kelly")
print("   • Higher Kelly = More conviction, but still manage risk")

## 5. Portfolio Allocation Strategy

### The Three-Tier Portfolio

Professional traders divide their portfolio into **three categories**:

#### 1. Core Holdings (40%)

**Purpose**: Stable, long-term positions

**Characteristics**:
- Blue-chip stocks
- Strong uptrends
- Low volatility
- Hold weeks to months

**Malaysian examples**:
- Banking: Maybank, Public Bank, CIMB
- Utilities: Tenaga Nasional
- Telecommunications: Maxis, Axiata

#### 2. Swing Trading (30%)

**Purpose**: Active trading, 3-15 day holds

**Characteristics**:
- Medium-term setups
- Technical analysis-based
- Clear entry/exit rules
- 3-15 day holding period

**Malaysian examples**:
- Construction: Gamuda, Sunway
- Plantation: Sime Darby Plantation, IOI Corp
- Healthcare: Top Glove, Hartalega

#### 3. Opportunistic (20%)

**Purpose**: High-conviction special situations

**Characteristics**:
- News-driven trades
- Earnings plays
- Sector rotation
- Quick in/out

**Malaysian examples**:
- Contract announcements (construction)
- Quarterly earnings beats
- Government policy changes

#### 4. Cash Buffer (10%)

**Purpose**: Safety and opportunity

**Why essential**:
- Handle margin calls
- Take advantage of opportunities
- Psychological comfort
- Pay brokerage fees

### Portfolio Allocation Example

**RM50,000 Account**:
- Core Holdings: RM20,000 (40%)
- Swing Trading: RM15,000 (30%)
- Opportunistic: RM10,000 (20%)
- Cash: RM5,000 (10%)

### Sector Diversification

**Never concentrate in one sector**:
- Maximum 30% in any single sector
- Spread across 3-5 sectors
- Consider correlation

**Malaysian sectors**:
- Banking & Finance
- Construction & Infrastructure
- Plantation & Agriculture
- Healthcare & Gloves
- Technology & Telecommunications

Let's build a diversified portfolio!

In [ ]:
# Portfolio allocation planner

def create_portfolio_allocation(account_size, core_pct=0.40, swing_pct=0.30, 
                               opp_pct=0.20, cash_pct=0.10):
    """
    Create three-tier portfolio allocation.
    
    Args:
        account_size (float): Total account value
        core_pct (float): Core holdings percentage
        swing_pct (float): Swing trading percentage
        opp_pct (float): Opportunistic percentage
        cash_pct (float): Cash buffer percentage
    
    Returns:
        dict: Portfolio allocation details
    """
    # Validate percentages sum to 100%
    total_pct = core_pct + swing_pct + opp_pct + cash_pct
    if abs(total_pct - 1.0) > 0.01:
        raise ValueError(f"Percentages must sum to 100% (currently {total_pct*100:.1f}%)")
    
    # Calculate allocations
    core_allocation = account_size * core_pct
    swing_allocation = account_size * swing_pct
    opp_allocation = account_size * opp_pct
    cash_allocation = account_size * cash_pct
    
    return {
        'account_size': account_size,
        'core': {
            'allocation': core_allocation,
            'percentage': core_pct * 100,
            'description': 'Long-term blue chips, weeks to months'
        },
        'swing': {
            'allocation': swing_allocation,
            'percentage': swing_pct * 100,
            'description': 'Active trading, 3-15 day holds'
        },
        'opportunistic': {
            'allocation': opp_allocation,
            'percentage': opp_pct * 100,
            'description': 'High-conviction special situations'
        },
        'cash': {
            'allocation': cash_allocation,
            'percentage': cash_pct * 100,
            'description': 'Safety buffer and opportunities'
        }
    }

# Example portfolio allocations
account_sizes = [10000, 25000, 50000, 100000]

print("Three-Tier Portfolio Allocation Strategy:")
print("=" * 80)

for account in account_sizes:
    portfolio = create_portfolio_allocation(account)
    
    print(f"\nAccount Size: RM{account:,}")
    print("-" * 80)
    print(f"{'Category':<18} {'Allocation':>15} {'%':>6} {'Description'}")
    print("-" * 80)
    
    for category, details in portfolio.items():
        if category != 'account_size':
            print(f"{category.title():<18} RM{details['allocation']:>13,.0f} "
                  f"{details['percentage']:>5.0f}% {details['description']}")

print("\n\n💡 Allocation Guidelines:")
print("   📊 Core (40%):        Low-risk blue chips, long-term holds")
print("   📈 Swing (30%):       Medium-risk active trading, 3-15 days")
print("   🎯 Opportunistic (20%): High-risk special situations")
print("   💵 Cash (10%):        Safety buffer, always maintain")

print("\n🇲🇾 Malaysian Market Example:")
print("   Core:         Maybank, Public Bank, Tenaga")
print("   Swing:        Gamuda, IOI Corp, Top Glove")
print("   Opportunistic: News-driven, earnings plays")
print("   Cash:         Ready for opportunities")

In [ ]:
# Visualize portfolio allocation

import matplotlib.pyplot as plt

# Create sample portfolio
sample_account = 50000
portfolio = create_portfolio_allocation(sample_account)

# Prepare data for pie chart
categories = []
allocations = []
colors = ['#2E86AB', '#A23B72', '#F18F01', '#06A77D']

for category, details in portfolio.items():
    if category != 'account_size':
        categories.append(f"{category.title()}\n({details['percentage']:.0f}%)")
        allocations.append(details['allocation'])

# Create pie chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Pie chart
ax1.pie(allocations, labels=categories, autopct='RM%1.0f',
       colors=colors, startangle=90, textprops={'fontsize': 11})
ax1.set_title(f'Portfolio Allocation\n(RM{sample_account:,} Account)',
             fontsize=14, fontweight='bold')

# Bar chart
bars = ax2.bar(range(len(categories)), allocations, color=colors, alpha=0.8)
ax2.set_xticks(range(len(categories)))
ax2.set_xticklabels([cat.split('\n')[0] for cat in categories], rotation=0)
ax2.set_ylabel('Allocation (RM)', fontsize=12)
ax2.set_title('Portfolio Allocation by Category', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, allocations)):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'RM{val:,.0f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Portfolio Allocation Principles:")
print("   • Never put all capital in one category")
print("   • Adjust percentages based on market conditions")
print("   • Rebalance monthly or quarterly")
print("   • Maintain cash buffer at all times")

## 6. Drawdown Management Protocol

### What is Drawdown?

**Drawdown** = Peak-to-trough decline in account value

**Example**:
- Account peak: RM50,000
- Current value: RM45,000
- Drawdown: RM5,000 or 10%

### Why Drawdown Management Matters

**Every trader experiences drawdowns**:
- Even profitable strategies have losing streaks
- Market conditions change
- Psychological pressure increases

**Without a protocol**:
- Panic selling
- Revenge trading
- Position size increases (trying to recover fast)
- Account destruction

### Drawdown Management Protocol

#### Level 1: 10% Drawdown (Yellow Alert)

**Actions**:
- [ ] Review all open positions
- [ ] Tighten stop-losses
- [ ] Reduce position sizes by 25%
- [ ] Only take highest-confidence setups
- [ ] Review recent trades for errors

#### Level 2: 15% Drawdown (Orange Alert)

**Actions**:
- [ ] Close all losing positions
- [ ] Reduce position sizes by 50%
- [ ] Only trade highest-conviction setups
- [ ] Take 3-5 day break from trading
- [ ] Review strategy thoroughly

#### Level 3: 20% Drawdown (Red Alert)

**Actions**:
- [ ] STOP TRADING immediately
- [ ] Close all positions
- [ ] Take 2-4 week break
- [ ] Complete strategy review
- [ ] Consider paper trading to regain confidence
- [ ] Seek mentorship/guidance

### Recovery Protocol

**After a drawdown**:
1. Start with reduced position sizes (50%)
2. Prove consistency for 10+ trades
3. Gradually increase to full size
4. Never try to "make it back fast"

### Malaysian Market Context

**Typical drawdowns on Bursa Malaysia**:
- 5-10%: Normal, don't overreact
- 10-15%: Review needed, adjust risk
- 15-20%: Serious, reduce activity
- 20%+: Stop and reset

Let's implement drawdown tracking!

In [ ]:
# Drawdown tracking and management system

def track_drawdown(account_history):
    """
    Calculate drawdown from account history.
    
    Args:
        account_history (list): List of account values over time
    
    Returns:
        dict: Drawdown statistics
    """
    df = pd.DataFrame({'value': account_history})
    
    # Calculate running maximum (peak)
    df['peak'] = df['value'].cummax()
    
    # Calculate drawdown
    df['drawdown'] = df['value'] - df['peak']
    df['drawdown_pct'] = (df['drawdown'] / df['peak']) * 100
    
    # Find maximum drawdown
    max_dd = df['drawdown'].min()
    max_dd_pct = df['drawdown_pct'].min()
    max_dd_idx = df['drawdown_pct'].idxmin()
    
    # Current drawdown
    current_dd = df['drawdown'].iloc[-1]
    current_dd_pct = df['drawdown_pct'].iloc[-1]
    
    return {
        'data': df,
        'current_value': df['value'].iloc[-1],
        'peak_value': df['peak'].iloc[-1],
        'current_drawdown': current_dd,
        'current_drawdown_pct': current_dd_pct,
        'max_drawdown': max_dd,
        'max_drawdown_pct': max_dd_pct,
        'max_drawdown_date': max_dd_idx
    }

def get_drawdown_action(drawdown_pct):
    """
    Determine action based on drawdown level.
    """
    if abs(drawdown_pct) < 10:
        return {
            'level': 'Normal',
            'color': '🟢',
            'action': 'Continue normal trading'
        }
    elif abs(drawdown_pct) < 15:
        return {
            'level': 'Yellow Alert (10-15%)',
            'color': '🟡',
            'action': 'Reduce position sizes 25%, review trades'
        }
    elif abs(drawdown_pct) < 20:
        return {
            'level': 'Orange Alert (15-20%)',
            'color': '🟠',
            'action': 'Reduce position sizes 50%, take 3-5 day break'
        }
    else:
        return {
            'level': 'Red Alert (20%+)',
            'color': '🔴',
            'action': 'STOP TRADING - Close all positions, take 2-4 week break'
        }

# Simulate account history with drawdown
# Realistic trading account with ups and downs
np.random.seed(42)

account_history = [10000]  # Start with RM10,000

# Simulate 100 days of trading
for i in range(100):
    # Random daily return between -3% and +3%
    daily_return = np.random.normal(0.001, 0.015)  # Slight positive bias
    new_value = account_history[-1] * (1 + daily_return)
    account_history.append(new_value)

# Calculate drawdown
dd_stats = track_drawdown(account_history)

print("Drawdown Analysis:")
print("=" * 80)
print(f"\nCurrent Account Value:  RM{dd_stats['current_value']:,.2f}")
print(f"Account Peak:           RM{dd_stats['peak_value']:,.2f}")
print(f"Current Drawdown:       RM{abs(dd_stats['current_drawdown']):,.2f} "
      f"({abs(dd_stats['current_drawdown_pct']):.2f}%)")
print(f"Maximum Drawdown:       RM{abs(dd_stats['max_drawdown']):,.2f} "
      f"({abs(dd_stats['max_drawdown_pct']):.2f}%)")

# Get recommended action
action = get_drawdown_action(dd_stats['current_drawdown_pct'])

print(f"\nDrawdown Level:         {action['color']} {action['level']}")
print(f"Recommended Action:     {action['action']}")

# Show recent account values
print("\nRecent Account History:")
print(f"{'Day':<6} {'Value':>12} {'Peak':>12} {'Drawdown %':>12}")
print("-" * 80)

recent_data = dd_stats['data'].tail(10)
for idx, row in recent_data.iterrows():
    print(f"{idx:<6} RM{row['value']:>10,.0f} RM{row['peak']:>10,.0f} {row['drawdown_pct']:>11.2f}%")

print("\n💡 Drawdown Management is Critical:")
print("   • Track drawdown daily")
print("   • Follow protocol strictly")
print("   • Don't revenge trade")
print("   • Preserve capital first")

In [ ]:
# Visualize drawdown over time

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)

dd_data = dd_stats['data']

# Top: Account value and peak
ax1.plot(dd_data.index, dd_data['value'], linewidth=2, label='Account Value',
        color='black', alpha=0.7)
ax1.plot(dd_data.index, dd_data['peak'], linewidth=2, label='Peak Value',
        color='green', linestyle='--', alpha=0.7)
ax1.fill_between(dd_data.index, dd_data['value'], dd_data['peak'],
                 where=dd_data['value'] < dd_data['peak'],
                 color='red', alpha=0.2, label='Drawdown')
ax1.set_title('Account Value and Drawdown Tracking', fontsize=16, fontweight='bold')
ax1.set_ylabel('Account Value (RM)', fontsize=12)
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Bottom: Drawdown percentage
ax2.fill_between(dd_data.index, 0, dd_data['drawdown_pct'],
                 color='red', alpha=0.6)
ax2.axhline(y=-10, color='yellow', linestyle='--', linewidth=2,
           label='Yellow Alert (-10%)', alpha=0.7)
ax2.axhline(y=-15, color='orange', linestyle='--', linewidth=2,
           label='Orange Alert (-15%)', alpha=0.7)
ax2.axhline(y=-20, color='red', linestyle='--', linewidth=2,
           label='Red Alert (-20%)', alpha=0.7)
ax2.set_title('Drawdown Percentage', fontsize=14, fontweight='bold')
ax2.set_xlabel('Trading Day', fontsize=12)
ax2.set_ylabel('Drawdown %', fontsize=12)
ax2.legend(loc='lower left', fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Visual Interpretation:")
print("   • Red shading: Periods in drawdown")
print("   • Dotted lines: Alert levels (10%, 15%, 20%)")
print("   • Drawdown is normal - protocol manages it")
print("   • Take action at alert levels, don't wait!")

## 7. Practice Exercises

Apply your position sizing and risk management knowledge!

### Exercise 1: Position Size Calculator

You have a RM25,000 account and want to trade the following setups. Calculate the correct position size for each:

1. **Maybank (1155.KL)**:
   - Entry: RM9.50
   - Stop: RM9.20 (3% stop)
   
2. **Gamuda (5398.KL)**:
   - Entry: RM3.80
   - Stop: RM3.60 (5.3% stop)
   
3. **Top Glove (7113.KL)**:
   - Entry: RM1.50
   - Stop: RM1.38 (8% stop)

For each trade:
- Calculate position size (2% risk)
- Calculate position value
- Verify actual risk amount
- Which trade allows the largest position? Why?

In [ ]:
# YOUR CODE HERE


### Exercise 2: ATR Position Sizing Comparison

Download recent data for three stocks:
- 1155.KL (Maybank)
- 5398.KL (Gamuda)
- 5225.KL (Top Glove)

For each stock:
1. Calculate current ATR (14-period)
2. Calculate position size using 2× ATR stop
3. Compare position sizes
4. Which stock gets the largest position? Why?
5. Does this make sense from a risk perspective?

In [ ]:
# YOUR CODE HERE


### Exercise 3: Kelly Criterion Application

Backtest results for your strategy show:
- Total trades: 50
- Winning trades: 32
- Losing trades: 18
- Average win: 4.5%
- Average loss: 2.8%

Calculate:
1. Win rate
2. Win/Loss ratio
3. Full Kelly percentage
4. 1/4 Kelly, 1/3 Kelly, 1/2 Kelly
5. Which fraction would you use and why?
6. Is this strategy worth trading based on Kelly?

In [ ]:
# YOUR CODE HERE


### Exercise 4: Portfolio Allocation Design

You have RM75,000 to invest in Malaysian stocks. Design a complete portfolio:

1. Allocate funds to Core/Swing/Opportunistic/Cash
2. Select specific stocks for each category:
   - Core: At least 3 blue chips
   - Swing: At least 2 active traders
   - Opportunistic: 1-2 special situations
3. Ensure sector diversification (max 30% per sector)
4. Calculate position sizes for each holding (use 2% rule)
5. Verify total allocation = RM75,000

Present your portfolio with:
- Stock ticker and name
- Category (Core/Swing/Opportunistic)
- Sector
- Position size
- % of portfolio

In [ ]:
# YOUR CODE HERE


## 8. Summary and Key Takeaways

Excellent work! You've mastered position sizing and risk management - the most critical skills for long-term trading success.

### ✅ Skills Mastered

1. **2% Rule**: Never risk more than 2% per trade
2. **Position Sizing**: Calculate size based on account and stop distance
3. **ATR Sizing**: Volatility-adjusted position sizing
4. **Kelly Criterion**: Optimal sizing based on strategy statistics
5. **Portfolio Allocation**: Three-tier system (Core/Swing/Opportunistic/Cash)
6. **Drawdown Management**: Protocol for handling losing periods
7. **Diversification**: Sector and position limits

### 📊 Key Formulas

**Position Size**:
```
Position Size = (Account Size × Risk %) / Stop Distance
```

**ATR Position Size**:
```
Stop Distance = ATR × Multiplier
Position Size = (Account Size × Risk %) / (ATR × Multiplier)
```

**Kelly Criterion**:
```
Kelly % = W - [(1 - W) / R]
Where W = Win Rate, R = Win/Loss Ratio
Use Fractional Kelly (1/4 or 1/3)
```

### 🇲🇾 Malaysian Market Guidelines

**Risk Per Trade**:
- Beginners: 1%
- Standard: 2%
- Never exceed: 3%

**Stop-Loss Guidelines**:
- Blue chips: 3-5%
- Mid caps: 5-7%
- Small caps: 7-10%

**Portfolio Allocation**:
- Core Holdings: 40% (Blue chips)
- Swing Trading: 30% (Active trades)
- Opportunistic: 20% (Special situations)
- Cash Buffer: 10% (Always maintain)

**Drawdown Protocol**:
- 10% drawdown: Yellow alert, reduce sizes 25%
- 15% drawdown: Orange alert, reduce sizes 50%
- 20% drawdown: Red alert, STOP TRADING

### ⚠️ Critical Rules

1. **ALWAYS calculate position size BEFORE entering**
2. **NEVER exceed 2-3% risk per trade**
3. **NEVER put more than 30% in one sector**
4. **ALWAYS maintain 10% cash buffer**
5. **FOLLOW drawdown protocol strictly**
6. **SIZE based on stop distance, not account size**
7. **WIDE stop = SMALLER position**
8. **RESPECT lot size minimums (100 shares)**

### 💡 Professional Insights

**Position sizing is MORE important than**:
- Entry timing
- Exit timing
- Indicator selection
- Technical analysis skill

**Why**:
- Proper sizing ensures survival
- Survival allows compounding
- Compounding creates wealth

**Amateur vs Professional**:
- Amateur: "How much can I buy?"
- Professional: "How much should I risk?"

### 🎯 What's Next?

In **Module 08: Backtesting Frameworks**, you'll learn:
- How to backtest strategies systematically
- Walk-forward testing methodology
- Performance metrics (Sharpe ratio, profit factor)
- Avoiding overfitting and curve-fitting
- Malaysian market transaction costs
- Realistic testing with slippage

### 📝 Risk Management Checklist

**Before Every Trade**:
- [ ] Account size confirmed
- [ ] Risk % decided (1-2%)
- [ ] Entry price determined
- [ ] Stop-loss placed
- [ ] Stop distance calculated
- [ ] Position size calculated
- [ ] Position value confirmed
- [ ] Risk amount verified (< 2% of account)
- [ ] Sector exposure checked (< 30%)
- [ ] Cash buffer maintained (≥ 10%)
- [ ] Ready to accept the loss if wrong

### 🎓 Self-Assessment

Before moving to Module 08, ensure you can:
- ✅ Explain why the 2% rule is essential
- ✅ Calculate position size for any trade setup
- ✅ Use ATR for volatility-adjusted sizing
- ✅ Apply Kelly Criterion appropriately
- ✅ Design a three-tier portfolio allocation
- ✅ Implement drawdown management protocol
- ✅ Understand that position sizing > entry timing

### 📚 Additional Resources

**Books**:
- *The Mathematics of Money Management* by Ralph Vince
- *Trade Your Way to Financial Freedom* by Van K. Tharp
- *Fortune's Formula* by William Poundstone (Kelly Criterion)

**Key Concept**:
*"Risk management is not about making money. It's about not losing money. If you don't lose money, making money becomes the easy part." - Professional Trading Wisdom*

---

**Congratulations on completing Module 07!** 🎉

You now understand that **position sizing determines 90% of your results** - not your entry timing.

**Next up**: `08_backtesting_frameworks.ipynb` - Learn to test your strategies systematically!

---

*"The goal of a successful trader is to make the best trades. Money is secondary." - Alexander Elder*